In [1]:
import numpy as np
import matplotlib.pyplot as plt
from MultinestClusterFit import MultinestClusterFit
from RebinningFunctions import rebin_data_exp
from PointingClusters import PointingClusters, save_clusters, load_clusters
from ModelSources import *

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances 

In [ ]:
# Example for that one orbit that required a second source to fit well

In [1]:
# Creates a folder in the local directory in which output files are saved
folder = "orbit_1019_two_sources"

# Creates the pointing combinations
# Replace "crab_data/1019" with the folder in which SPI orbit data are saved, but be sure to enter it as a tuple
# Required are the files "dead_time.fits", "energy_boundaries.fits", "evts_det_spec.fits", "pointing.fits"
# Multiple orbits should also work, but I haven't tested this much
# To use pointing clusters of size three, set cluster_size_range=(3,3)
# Other parameters should be fine as is, and self-explantory to change
pointings = PointingClusters(
    ("crab_data/1019",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_crab=10.,
    min_time_elapsed=600.,
    cluster_size_range=(2,2),
).pointings

# This saves and loads the pointing clusters
# It is useful if you want to fit the same pointing clusters several times without having to regenerate them every time
save_clusters(pointings, folder)
pointings = load_clusters(folder)

# Add or remove any sources to be fitted
# See "ModelSources" for more sources and which parameters they require
source_model = define_sources((
    (crab_pl_fixed_pos, (40,)),
    (s_1A_0535_262_pl, (40,)),
))

# Does the fit
multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (None, 80), # Energy Range
    np.geomspace(18, 150, 50), # Energy bins of source
    rebin_data_exp, # Binning function for counts
    true_values=true_values(), # True values for reference
    folder=folder,
)

# Plots the paramter distributions
# See line 357 in "MultinestClusterFit" to edit
# You may also want to see line 198, where the ChainConsumer object is initialized
multinest_fit.parameter_fit_distribution()

# Creates text files with useful information of the fit
multinest_fit.text_summaries()

# Creates PPC plots
# In "MultinestClusterFit" see
# line 526 to edit count_energy_plots
# line 567 to edit qq_plots
# line 607 to edit rel_qq_plots
# cdf_hists are still a work in progress and is not recommended
multinest_fit.ppc(
    count_energy_plots=True,
    qq_plots=True,
    rel_qq_plots=True,
    cdf_hists=False
)

In [ ]:
# Example for an orbit which works well for only the crab, using a Crab source with
# non-fixed position

In [2]:
folder = "orbit_1644_movable_crab"

pointings = PointingClusters(
    ("crab_data/1644",),
    min_angle_dif=1.5,
    max_angle_dif=7.5,
    max_time_dif=0.2,
    radius_around_crab=10.,
    min_time_elapsed=600.,
    cluster_size_range=(2,2),
).pointings
save_clusters(pointings, folder)

pointings = load_clusters(folder)
source_model = define_sources((
    (crab_pl_free_pos, (40,)),
))

multinest_fit = MultinestClusterFit(
    pointings,
    source_model,
    (20., 81.5),
    np.geomspace(18, 150, 50),
    rebin_data_exp,
    true_values=true_values(),
    folder=folder,
)

multinest_fit.parameter_fit_distribution()
multinest_fit.text_summaries()
multinest_fit.ppc(
    count_energy_plots=True,
    qq_plots=True,
    rel_qq_plots=True,
    cdf_hists=False
)